In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_table('sdrf (3).tsv')

In [3]:
df['comment[instrument]'] = 'Q Exactive'
df['comment[technical replicate]'] = 1
df['characteristics[organism]'] = 'Homo Sapiens'
df['characteristics[organism part]'] = 'breast'
df['comment[fraction identifier]'] = df['comment[data file]'].str.extract(r'fr(\d+).raw').astype(int)
df['comment[file uri]'] = 'https://storage.jpostdb.org/JPST000265/' + df['comment[data file]']
df['technology type'] = 'proteomic profiling by mass spectrometry'
df['characteristics[ancestry category]'] = 'not available'
df['characteristics[age]'] = 'not available'
df['characteristics[sex]'] = 'female'
df['characteristics[cell type]'] = 'malignant cell'
df['characteristics[biological replicate]'] = 1

In [4]:
tumor_id = pd.read_excel('41467_2019_9018_MOESM3_ESM.xlsx', sheet_name='Tumor annotations', usecols=['Tumor ID', 'TMT set nr', 'TMT tag', 'PAM50 subtype'], index_col=(1, 2))

In [5]:
tumor_id.index = pd.MultiIndex.from_tuples([(s, str(label)) for s, label in tumor_id.index], names=tumor_id.index.names)

In [6]:
tumor_types = {
    'Basal': 'basal-like breast carcinoma',
    'LumA': 'luminal A breast carcinoma',
    'LumB': 'luminal B breast carcinoma',
    'HER2': 'HER2 Positive Breast Carcinoma',
    'Normal': 'Normal Breast-Like Subtype of Breast Carcinoma'
}

In [7]:
tumor_id.head()

Tumor ID PAM50 subtype
TMT set nr TMT tag                       
1          126      OSL.53E         Basal
           127N     OSL.567          LumA
           127C     OSL.3FF         Basal
           128N     OSL.55F         Basal
           128C     OSL.46A         Basal

In [8]:
pool_str = 'SN=' + ','.join(tumor_id['Tumor ID'].values)

def get_info(row):
    pool = int(re.search(r'pool(\d)', row['comment[data file]']).group(1))
    try:
        sample = tumor_id.loc[(pool, row['comment[label]'][3:]), 'Tumor ID']
        disease = tumor_types[tumor_id.loc[(pool, row['comment[label]'][3:]), 'PAM50 subtype']]
        pooled = 'not pooled'
    except KeyError:
        sample = 'pool'
        disease = 'breast cancer'
        pooled = pool_str
    assay = f"pool {pool}, fraction {row['comment[fraction identifier]']}"
    return sample, assay, pooled, disease

In [9]:
df[['source name', 'assay name', 'characteristics[pooled sample]', 'characteristics[disease]']] = df.apply(get_info, axis=1, result_type='expand')
df['factor value[disease]'] = df['characteristics[disease]']

In [10]:
def key(colname):
    if colname == 'source name':
        return 0
    if colname[:15] == 'characteristics':
        return 1
    if colname == 'assay name':
        return 2
    if colname == 'technology type':
        return 3
    if colname[:7] == 'comment':
        return 4
    if colname[:12] == 'factor value':
        return 5
    return 6

In [11]:
df = df[sorted(df.columns, key=key)]

In [12]:
df.to_csv('PXD008841.sdrf.tsv', sep='\t', index=False)